# Creating binary images: 

- Before an image can be segmented a binary image is needed 
- To do this we use Otsu thresholding which finds a cut-off value for background and foreground. 
- This number is then used in the next notebooks to find porosity values

### Create a file for the threhsold values for each sample/tower

In [7]:
import os

os.listdir('../../')


['.git',
 '.gitattributes',
 '20250312_PorosityAnalysis',
 'data',
 'notebooks',
 'README.md']

In [8]:
# Set up the module for the porosity_analysis packages  

import os 
import sys 

Pyro_module  = '../../20250312_PorosityAnalysis/porosity_analysis'
# Add the parent directory of Pyro_DataAnalysis to the system path
if os.path.exists(Pyro_module) == True:
    sys.path.insert(0, os.path.abspath(Pyro_module))
else :
    print('The file does not exist')

In [19]:
#import statements 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
from pathlib import Path

from skimage.filters import gaussian
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops,regionprops_table ,find_contours
from skimage.color import label2rgb
import matplotlib.patches as mpatches
import imageio.v3 as iio

import config 
import custom_funcs 
import image_analysis as ia

In [10]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# to make sure the computer doesnt think you are deploying a bomb 
import PIL.Image
PIL.Image.MAX_IMAGE_PIXELS = 3019898880 

In [20]:
# Google drive location or location of the image folder
experimental_dir = Path(config.technical_dir_path)

In [ ]:
# Save location
df_save_loc = experimental_dir/'data/processed/global_threshold.csv'

In [22]:
## Check what has been done already: 

if os.path.exists(df_save_loc) == True:
    df_done = pd.read_csv(df_save_loc)
    ref_im_done = list(df_done['ref_im'])
else:
    df_done = pd.DataFrame(columns= ['ref_im','date','Material','Designation','Otsu']) # Create Dataframe with the column headings
    ref_im_done = list(df_done['ref_im'])
print(f'These files have been done :{ref_im_done}')

These files have been done :[]


### Pre-processing in Fiji

#### Before anything can happen:
- The images should be cropped to just the relevant area
- Saved all images in .png in the processed folder 
- Put full images for thresholding in thresholding folder

### Reference image for thresholding 

In [ ]:
ref_im_dir_temp = experimental_dir/'Raw/png' # the place all the raw images 
ref_im_dir = [file for file in os.listdir(ref_im_dir_temp) if file.endswith('.png')] # look only at .png 
ref_im_dir

[]

In [31]:
df = pd.DataFrame(columns= ['ref_im','date','Material','Designation','Otsu']) # Create Dataframe with the column headings for results

In [53]:

for ref_im in os.listdir(ref_im_dir):
    parts = ref_im.split("_")

    if ref_im in ref_im_done:
        continue
    
    if len(parts) >= 3: #check if the filename is in the correct format.
            date = parts[0]
            Material = parts[1]
            Designation = parts[2]

            # load image
            ref_img = os.path.join(ref_im_dir, ref_im)
            
            im = custom_funcs.load_image(ref_img)
            ## check if an image is greyscale
            gs_img = custom_funcs.is_greyscale(im)
            
            # Clean image 
            image_blur = gaussian(gs_img,2)
            # Get threshold value 
            Otsu = threshold_otsu(image_blur)
            after_Otsu = image_blur < Otsu


            # Append to DataFrame
            df = pd.concat([df, pd.DataFrame([{'ref_im': ref_im, 'date': date, 'Material': Material, 'Designation': Designation, 'Otsu': Otsu}])], ignore_index=True)
            



C:\Users\Neil\AppData\Local\Temp\ipykernel_14768\2015535755.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([{'ref_im': ref_im, 'date': date, 'Material': Material, 'Designation': Designation, 'Otsu': Otsu}])], ignore_index=True)


### Create df 

In [59]:
df.to_csv(df_save_loc, index=False, header=False, mode='a')

In [32]:
df

,ref_im,date,Material,Designation,Otsu
